# TEST - TECH Engines – Junior Data Scientist

## Objective

The objective is to implement this formula:

$$
\mathcal{L}(S,R,U,T,F,G) = 
\dfrac{1}{2}||\mathcal{A} - \mathcal{S} \times_{R} R \times_{U} U \times_{T} T||^{2} 
+ \dfrac{\lambda_{1}}{2}||X - TG||^{2} 
+ \dfrac{\lambda_{2}}{2}||Y - RF||^{2}
+ \dfrac{\lambda_{3}}{2}(||\mathcal{S}||^{2} + ||R||^{2} + ||U||^{2} + ||T||^{2} + ||F||^{2} + ||G||^{2}) \, .
$$

This function is called the **objective function** and it is the equivalent of the cost function in machine learning: it is used in an optimization algorithm with the objective of filling the entries of $\mathcal{A}$, which is a sparse 3D array of data. Before going into the detail of the formula let's focus on each variable.

## Variable Description

Let's look at the variables in the objective function formula $\mathcal{L}$:

- $\mathcal{A}$  is a **sparse 3D array of dimensions** $\rm d_1\!\times d_2\times d_3$. This is a **matrix of data**, where a lot of its entries are missing. 

    **Note:** The final objective for which the function $\mathcal{L}$ is used is to fill in the entries of $\mathcal{A}$!


- $X$ are $Y$ are **2D arrays**. They are **matrices of data**, which should not be modified by program, and that I think should be **global variables**.
    * $X$ has dimensions $\rm d_3\times d_X$
    * $Y$ has dimensions $\rm d_1 \times d_Y$


- $R$, $U$, $T$, $F$ and $G$ are **2D arrays**. These matrices are used in the optimization algorithm: they are initialized with small random values and then updated at every iteration to minimize the $\mathcal{L}$ function.
    * $R$ has dimensions $\rm d_1 \times d_R$
    * $U$ has dimensions $\rm d_2 \times d_U$
    * $T$ has dimensions $\rm d_3 \times d_T$
    * $F$ has dimensions $\rm d_R \times d_Y$
    * $G$ has dimensions $\rm d_T \times d_X$


- $\mathcal{S}$ is a **3D array of dimensions** $\rm d_R \times d_U \times d_T$. This is also used in the optimization algorithm and it is updated every iteration to minimize the $\mathcal{L}$ function.


- $\lambda_1$, $\lambda_2$ and $\lambda_3$ are **parameters** of the function, which will have to be tuned during the hyperparameter tuning.

### Note on Dimensions

Dimensions are important as they are correlated!

- $d_1$, $d_2$, $d_3$, $d_X$ and $d_Y$ are in general very large, and directly depend on the data.
- $d_R$, $d_U$ and $d_T$ are usually small, and are fixed by the program.

## Decoding the Formula

Now, let's look at the formula step by step, and see what each term means:
 - $||\cdot ||$ is the square root of the sum of the squared entries.
     
    In Python this norm can be implemented in the following way:

In [6]:
import numpy as np

def norm(A):
    sum = 0
    for a in A.flatten():
        sum += a**2
    norm = np.sqrt(sum)
    return norm

   or using the `scipy` function `lingalg.norm()`, which is faster:

In [7]:
from scipy import linalg

# Create a 2D array and a 3D array
np.random.seed(1)
A = np.random.rand(5,7)*10
B = np.random.rand(5,7,2)*10

# Calculating the norm with linalg.norm()
norm_A = linalg.norm(A)
norm_B = linalg.norm(B)

# Calculating the norm with the norm() function
my_norm_A = norm(A)
my_norm_B = norm(B)

# Check is the norm() function gives the same result as the linalg.norm()
print(format(my_norm_A, '1.14f') == format(norm_A, '1.14f'))
print(format(my_norm_B, '1.14f') == format(norm_B, '1.14f'))

True
True


- $TG$ and $RF$ are matrix multiplications, which in Python are implemented by the `np.dot()` function.

- $\times_R$ denotes a multiplication between a 3D array and a 2D array which results in a new 3D array. The suffixes just denote the direction of the multiplication. 
I did not find a built-in function in Python that does this, but it might be worth it to keep looking. As I implemented this, I thought that the best thing to do was to calculate the whole term $\mathcal{S} \times_{R} R \times_{U} U \times_{T} T$:

In [8]:
def tensor_matrix_dot(T,M1,M2,M3):
    """
    type T: 3D array
    type M1: 2D array
    type M2: 2D array
    type M3: 2D array
    rtype: 3D array
    NOTE: the order of the matrices is important! It is such that 
    T.shape[0]=M1.shape[1]
    T.shape[1]=M2.shape[1]
    T.shape[2]=M3.shape[1]
    """
    # Initialize the 3D array where we store the results
    W = np.zeros((M1.shape[0], M2.shape[0], M3.shape[0]))
    
    # Compute the entries of A
    for i in range(M1.shape[0]):
        for j in range(M2.shape[0]):
            for k in range(M3.shape[0]):
                for l in range(M1.shape[1]):
                    for m in range(M2.shape[1]):
                        for n in range(M3.shape[1]):
                            W[i,j,k] += T[l,m,n]*M1[i,l]*M2[j,m]*M3[k,n]
    
    return W

**Note:** This implementation needs improvement. The matrices that are going to be used are very large and the moltitude of for loops will make the computation of this product very costly.

The image below shows how this product of a 3D array and 3 2D arrays gives a 3D array:

<img src="tensor_matrix_dot2.jpg"  width='300'>

## Example of implementation of the function 

Putting everything together, here is an example of the implementation of the function.

In [9]:
import numpy as np
from scipy import linalg

np.random.seed(123)

# Create some sample data
# A: 3D array d1*d2*d3
# X: 2D array d3*dX
# Y: 2D array d1*dY
A = np.random.rand(15,10,24)*100 # NOTE: in a real-case scenario this is sparse!!
X = np.random.rand(24,16)*100
Y = np.random.rand(15,8)*100

# Dimensions
(d1, d2, d3) = A.shape
dX = X.shape[1]
dY = Y.shape[1]
dR, dU, dT = 5, 5, 5

# Initialize S, R, U, T, F and G with small random values
S = np.random.rand(dR,dU,dT)
R = np.random.rand(d1,dR)
U = np.random.rand(d2,dU)
T = np.random.rand(d3,dT)
F = np.random.rand(dR,dY)
G = np.random.rand(dT,dX)


def L(S, R, U, T, F, G, l1=1, l2=1, l3=1):
    """
    :type S: 3D array dR*dU*dT
    :type R: 2D array d1*dR
    :type U: 2D array d2*dU
    :type T: 2D array d3*dT
    :type F: 2D array dR*dY
    :type G: 2D array dT*dX
    :type l1: float
    :type l2: float
    :type l3: float
    :rtype: float
    """
    # Calculate each term of the formula
    normA2 = linalg.norm(A - tensor_matrix_dot(S,R,U,T))**2
    normX2 = linalg.norm(X - np.dot(T,G))**2
    normY2 = linalg.norm(Y - np.dot(R,F))**2
    normS2 = linalg.norm(S)**2
    normR2 = linalg.norm(R)**2
    normU2 = linalg.norm(U)**2
    normT2 = linalg.norm(T)**2
    normF2 = linalg.norm(F)**2
    normG2 = linalg.norm(G)**2
    
    # Calculate the result of the formula
    result = normA2/2 + l1/2*normX2 + l2/2*normY2 + l3/2*(normS2 + normR2 + normU2+ normT2 + normF2 + normG2)
    
    return result

In [10]:
L(S,R,U,T,F,G)

5537743.681647928

By just using random numbers this number is very big!

## Bigger Picture

After implementing the function $\mathcal{L}$, the final objective would be to use it inside an optimization algorithm which updates the values of $\mathcal{S}$, $R$, $U$, $T$, $F$ and $G$ at every iteration to minimize the output of the function $\mathcal{L}$. Once this is done, one can use $\mathcal{S}$, $R$, $U$ and $T$ to fill in the missing entries of $\mathcal{A}$ like this:
$$
\mathcal{A}_{fill} = \mathcal{S} \times_{R} R \times_{U} U \times_{T} T
$$
or, using the example of implementation of this product: `tensor_matrix_dot(S,R,U,T)`.

##  Applications and Further Resources

The applications of this function are varied. As an example, the authors of the [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/frp0557-zheng1.pdf) we are refferring to use it to estimate the travel time of any path in a city in real time, starting from just a few GPS trajectories of vehicles received in current time (which is their sparce $\mathcal{A}$ matrix) and a period of history data (which is their $X$) as well as map data sources (which is their $Y$).

Another good resource is this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/frp0557-zheng1.pdf) where they have released the [data and the code](https://www.microsoft.com/en-us/research/publication/travel-time-estimation-of-a-path-using-sparse-trajectories/?from=http%3A%2F%2Fresearch.microsoft.com%2Fapps%2Fpubs%2F%3Fid%3D217493), which might be a good future reference in case we want to develope the optimization algorithm.